In [1]:
from libs.plugin_converter.semantic_kernel_v0_to_openai_function import generate_callables, generate_definitions, make_context
from libs.plugins.plugin_capital import PluginCapital
from libs.plugins.plugin_salesforce import PluginSalesforce
from libs.plugins.plugin_sap import PluginSAP
from libs.plugins.plugin_capivara import PluginCapivaraAlert
import libs.plugin_converter.semantic_kernel_v0_to_openai_function as semantic_kernel_v0_to_openai_function
import libs.plugin_converter.openai_function_to_tool as openai_function_to_tool

import os
from libs.utils.connector_llm import *
from libs.plugins.plugin_capital import PluginCapital
import libs.plugin_converter.semantic_kernel_v0_to_openai_function as semantic_kernel_v0_to_openai_function
import libs.plugin_converter.openai_function_to_tool as openai_function_to_tool
from libs.plugin_orchestrator.implementation_tool import OrchestratorWithTool
from libs.plugin_orchestrator.implementation_bare import OrchestratorBare

import dotenv
dotenv.load_dotenv()

True

In [2]:
#semantic_kernel_v0_to_openai_function.generate_callables([(PluginSAP(), "PluginSAP")])

In [3]:
# Bare openai
#llm = factory_create_connector_llm(
#    provider='azure_openai',
#    modelname=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME_CHAT"),
#    version=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME_CHAT").split('-')[-1],
#    credentials=CredentialsOpenAI(
#        base_url=os.getenv("AZURE_OPENAI_ENDPOINT"),
#        api_key=os.getenv("AZURE_OPENAI_KEY"),
#    ),
#    hyperparameters={'tool_choice': 'none'}
#)

# Tool openai
llm = factory_create_connector_llm(
    provider='azure_openai',
    modelname=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME_CHAT"),
    version=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME_CHAT").split('-')[-1],
    credentials=CredentialsOpenAI(
        base_url=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_KEY"),
    ),
)


selected_alert = {'id': 'dc2a',
 'title': 'Defective product',
 'description': 'I ordered a Handstückschlauch, but it came already broken',
 'severity': 'Medium'}

plugins = [
    (PluginCapivaraAlert(capivara_base_url='http://capivara:80', alert_id=selected_alert['id']), "PluginServiceNow"),
    (PluginSalesforce(), "PluginSalesforce"),
    (PluginSAP(), "PluginSAP"),
]
orchestrator = OrchestratorWithTool(
    connection = llm,
    tool_definitions = openai_function_to_tool.generate_definitions(semantic_kernel_v0_to_openai_function.generate_definitions(plugins)),
    tool_callables = generate_callables(plugins),
    token_limit_input = 1024,
    token_limit_output = None,
    max_steps_recommended = 4,
    max_steps_allowed = 5,
    prompt_app_system=(
        'You are an AI assistant whose goal is to help the use handle a support ticket.\n'
        'Here is some info about this ticket:\n'
        f'Title: {selected_alert["title"]}\n'
        f'Description: {selected_alert["description"]}\n'
        f'Severity: {selected_alert["severity"]}\n'
        'Answer the user questions and help the user to resolve this ticket.\n'
    ),
    prompt_app_user='hi =D'  # chitchat
    #prompt_app_user='What purchases have this customer done in the past?',  # SAP
    #prompt_app_user='place an order for 10 gloves',  # SAP
    #prompt_app_user='If the customer had purchased any glove in the past, please place an order for 100 of that exact glove model',  # SAP
    #prompt_app_user='IF the customer purchased any hand desinfectant in the past, create an appointment for a sales call and describing we may want to do a follow up sales',  # SAP
    #prompt_app_user='what is the full name of this customer?',  # Salesforce
    #prompt_app_user='create an appointment asking an employee to visit the customer facility, and don\'t forget to mention the customer address',  # Salesforce, SAP
    #prompt_app_user='place an order for 10 gloves, then close the current ticket',  # SAP, capivara
)

r = await orchestrator.run()
print(r)
print('\n\n' + '-'*30 + '\n\n')
assert type(r.answer) == str
assert len(r.answer) > 4
print(r.answer)

[2024-08-20 12:48:07 +0000] libs.libs.plugin_orchestrator INFO     Function calling detected by: native API feature
[2024-08-20 12:48:07 +0000] libs.libs.plugin_orchestrator INFO     At step 1 the function PluginSalesforce_get_account_details was requested, with args = {}
[2024-08-20 12:48:07 +0000] libs.libs.plugin_orchestrator INFO     result: Executed function PluginSalesforce_get_account_details and returned: {"name": "John Doe", "email": "john.d@something.com", "phone": "123-456-7890", "address": "1234 Elm St, Springfield, IL 62701"}
[2024-08-20 12:48:07 +0000] libs.libs.plugin_orchestrator INFO     Function calling detected by: native API feature
[2024-08-20 12:48:07 +0000] libs.libs.plugin_orchestrator INFO     At step 2 the function PluginSAP_create_service_appointment was requested, with args = {'description': 'Defective Handstückschlauch'}
[2024-08-20 12:48:07 +0000] libs.libs.plugin_orchestrator INFO     result: Executed function PluginSAP_create_service_appointment and retu

In [ ]:
#############################################
# Dataset generation
orchestrator._full_message_history_for_debugging
# This is the expected chat-over-text format:
# <|system|> You are a chatbot who can help code!</s> <|user|> Write me a function to calculate the first 10 digits of the fibonacci sequence in Python and print it out to the CLI.</s> <|assistant|>
# Before finetunning, we have to convert
# this format is called "Zephyr", because it is used in the model HuggingFaceH4/zephyr-7b-alpha

In [ ]:
'''# Bare llamacpp
llm = factory_create_connector_llm(
    provider='llama_cpp',
    modelname='not-needed',
    version='not-needed',
    credentials=CredentialsOpenAI(
        base_url='http://llm-server:8080/v1',
        api_key='not-needed',
    )
)


plugins = [
    (PluginCapital(), "PluginCapital"),
]
orchestrator = OrchestratorWithTool(
    connection = llm,
    tool_definitions = openai_function_to_tool.generate_definitions(semantic_kernel_v0_to_openai_function.generate_definitions(plugins)),
    tool_callables = generate_callables(plugins),
    token_limit_input = 1024,
    token_limit_output = None,
    max_steps_recommended = 2,
    max_steps_allowed = 3,
    prompt_app_system='You are an AI assistant whose goal is to answer the user question.',
     prompt_app_user='What is the capital of france?',
)

r = await orchestrator.run()
print(r)
print('\n\n' + '-'*30 + '\n\n')
assert type(r.answer) == str
assert len(r.answer) > 4
print(r.answer)''';